In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import copy
import matplotlib as mpl
from random import sample
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, fbeta_score, make_scorer, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve, validation_curve
from sklearn.utils.class_weight import compute_class_weight

plt.style.use('bmh')
mpl.rcParams.update({
    "grid.linestyle" : "dashed",
    "axes.facecolor" : "white",
    "axes.spines.top" : False,
    "axes.spines.right" : False,
    "legend.frameon" : False,
    "figure.figsize" : (8, 5),
    "figure.dpi" : 300,
})
%matplotlib inline

# Suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(42)

In [2]:
suffix_old = ""
suffix = ""

n_features = 29
path_thyroid = "data/thyroid/"
path_without_thyroid = "data/no_thyroid/"
path = path_without_thyroid if n_features == 18 else path_thyroid
path_models = f"models/{n_features}features/"

In [5]:
# Read data
df_train = pd.read_csv(f"{path}train{suffix}.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid{suffix_old}.csv", index_col=0)
df_test = pd.read_csv(f"{path}test{suffix_old}.csv", index_col=0)

if n_features == 7:
    top_variables = [
        "Dyslipidemia\nHystory of dyslipidemia",
        "fe",
        "Previous CABG",
        "Diabetes\nHistory of diabetes",
        "Previous Myocardial Infarction",
        "Smoke\nHistory of smoke",
        "Documented resting \nor exertional ischemia",
        "Survive7Y"
    ]
    df_train = df_train.loc[:, top_variables]
    df_valid = df_valid.loc[:, top_variables]
    df_test = df_test.loc[:, top_variables]

print(df_train)
train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()

# y_**** contains the value of Survive7y as a list
# X_**** contains everything except for Survive7y as a list of list
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

feat_names = list(df_train.columns)

# Print how Survive7y are distribuited in each set
from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

# All the numerical features that can be standardized
from utils import get_preprocess_std_num
preprocess_std = get_preprocess_std_num(feat_names)

preprocess_std_all = StandardScaler()

# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)
print(X_train_std)

        Gender (Male = 1)        Age   TSH   fT3    fT4  Euthyroid  \
Number                                                               
1567                    1  15.561798  3.20  3.73  11.73          1   
7443                    1  64.719101  2.38  2.77  17.20          1   
1427                    1  71.502809  1.08  2.42   9.21          1   
7953                    1  19.196629  1.80  3.60  14.30          1   
2835                    0  77.157303  1.54  1.53  16.30          0   
...                   ...        ...   ...   ...    ...        ...   
7516                    1  78.098315  2.06  2.84  11.50          1   
292                     1  63.980337  0.74  2.65   9.10          1   
6842                    1  69.564607  2.10  2.72   9.20          1   
4081                    1  93.410112  2.34  1.29  16.80          0   
70                      0  73.323034  4.55  2.36  10.40          0   

        Subclinical primary hypothyroidism (SCH)  \
Number                               

### Training


In [6]:
from functools import partial
from train import report, evaluate, train_and_evaluate

train_partial = partial(
    train_and_evaluate, 
    preprocess_std, 
    X_train=X_train,
    y_train=y_train,
    X_valid=X_valid,
    y_valid=y_valid,
    scoring="f1_macro", 
    iter=5000, 
    save=True,
    path_models = path_models
)

In [7]:
from sklearn.linear_model import LogisticRegression

hyperparams = {
    'model__penalty': ['l1', 'l2', 'elasticnet'], #type of penalities added, 'elasticnet' means both
    'model__dual': [True, False], # use primal or dual form. Default is True.  
    'model__warm_start': [True, False], # default is False. When set to True, reuse the solution of the previous call to fit as initialization, otherwise, just erase the previous solution. 
    'model__C': stats.randint(1, 10), #default is 1, if the value is larger, then it indicates stronger regularization
    'model__max_iter': stats.randint(50, 500), # Default is 100, Maximum number of iterations taken for the solvers to converge.
    'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], #default is lbfgs. 
}
#Default is None (thus weight = 1). Balanced uses the formula n_samples / (n_classes * np.bincount(y))
model = LogisticRegression(class_weight="balanced")
train_partial(model=model, hyperparams=hyperparams, savename="lr")
# metrics.plot_roc_curve(pipe, X_valid, y_valid)

# import math
# w = logreg.coef_[0]
# feature_importance = pd.DataFrame(df_feat.columns[:-1], columns=["features"])
# feature_importance["importance"] = pow(math.e, w)
# feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
# feature_importance

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits


/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/linea

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.396     0.606     0.479       505
         1.0      0.938     0.866     0.901      3494

    accuracy                          0.833      3999
   macro avg      0.667     0.736     0.690      3999
weighted avg      0.870     0.833     0.848      3999

auc macro 0.840
confusion matrix
[[ 306  199]
 [ 467 3027]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.384     0.580     0.462       169
         1.0      0.934     0.865     0.898      1165

    accuracy                          0.829      1334
   macro avg      0.659     0.723     0.680      1334
weighted avg      0.865     0.829     0.843      1334

auc macro 0.821
confusion matrix
[[  98   71]
 [ 157 1008]]
Model rank: 1
Mean validation score: 0.695 (std: 0.007)
Parameters: {'model__C': 5, 'model__dual': True, 'model__max_iter': 207, 'model__penalty': 'l2', 'model__solver': 'lib

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 LogisticRegression(C=5, class_weight='balanced', dual=True,
                                    max_iter=207, solver='liblinear',
                                    warm_start=True))])

In [8]:
from sklearn.svm import SVC

hyperparams = {
    'model__C': stats.randint(100, 600),
    'model__kernel': ['rbf', 'poly', 'sigmoid'],
    'model__degree': stats.randint(5, 200),
    'model__gamma': ['scale', 'auto'], #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. scale = 1 / (n_features * X.var()) as value of gamma,
    'model__coef0': stats.uniform(0.0, 1), # It is only significant in ‘poly’ and ‘sigmoid’.
    'model__max_iter': [400, 800, 1200, 1600]
}

model = SVC(class_weight="balanced", probability=True)
train_partial(model=model, hyperparams=hyperparams, savename="svc")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits


/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.219     0.483     0.302       505
         1.0      0.910     0.752     0.823      3494

    accuracy                          0.718      3999
   macro avg      0.565     0.617     0.562      3999
weighted avg      0.822     0.718     0.757      3999

auc macro 0.646
confusion matrix
[[ 244  261]
 [ 868 2626]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.213     0.473     0.294       169
         1.0      0.907     0.747     0.819      1165

    accuracy                          0.712      1334
   macro avg      0.560     0.610     0.557      1334
weighted avg      0.819     0.712     0.753      1334

auc macro 0.644
confusion matrix
[[ 80  89]
 [295 870]]
Model rank: 1
Mean validation score: 0.667 (std: 0.001)
Parameters: {'model__C': 106, 'model__coef0': 0.25736872987823345, 'model__degree': 32, 'model__gamma': 'scale', 'model__k

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 SVC(C=106, class_weight='balanced', coef0=0.25736872987823345,
                     degree=32, max_iter=1600, probability=True))])

In [9]:
from sklearn.neighbors import KNeighborsClassifier

hyperparams = {
    'model__n_neighbors': stats.randint(2, 100),
    'model__weights': ('uniform', 'distance'), # ‘distance’ : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.
    'model__algorithm': ('ball_tree', 'kd_tree'),
    'model__leaf_size': stats.randint(10, 60)
}

model = KNeighborsClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="knn")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.682     0.543     0.604       505
         1.0      0.936     0.963     0.949      3494

    accuracy                          0.910      3999
   macro avg      0.809     0.753     0.777      3999
weighted avg      0.904     0.910     0.906      3999

auc macro 0.936
confusion matrix
[[ 274  231]
 [ 128 3366]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.412     0.278     0.332       169
         1.0      0.900     0.942     0.921      1165

    accuracy                          0.858      1334
   macro avg      0.656     0.610     0.626      1334
weighted avg      0.838     0.858     0.846      1334

auc macro 0.698
confusion matrix
[[  47  122]
 [  67 1098]]
Model rank: 1
Mean validation score: 0.629 (std: 0.003)
Parameters: {'model__algorithm': 'ball_tree', 'model

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 KNeighborsClassifier(algorithm='ball_tree', leaf_size=11,
                                      n_neighbors=4))])

In [10]:
from sklearn.ensemble import RandomForestClassifier

hyperparams = {
    'model__n_estimators': stats.randint(10, 200),
    'model__criterion': ('gini', 'entropy'), # The function to measure the quality of a split. Tree-specific parameter
    'model__min_samples_split': stats.randint(1, 8), #The minimum number of samples required to split an internal node
    'model__min_samples_leaf': stats.randint(1, 5), # The minimum number of samples required to be at a leaf node
    'model__max_features': ('sqrt', 'log2', None), # If “sqrt”, then max_features=sqrt(n_features). If “log2”, then max_features=log2(n_features), If None, then max_features=n_features.
    'model__class_weight': ['balanced', 'balanced_subsample'], 
}

model = RandomForestClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="rf")

# feature importance use permutation importance
# importance = rf_rand.best_estimator_["model"].feature_importances_
# plt.bar(list(range(len(importance))), importance)

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.807     0.954     0.875       505
         1.0      0.993     0.967     0.980      3494

    accuracy                          0.965      3999
   macro avg      0.900     0.961     0.927      3999
weighted avg      0.970     0.965     0.967      3999

auc macro 0.992
confusion matrix
[[ 482   23]
 [ 115 3379]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.424     0.379     0.400       169
         1.0      0.911     0.925     0.918      1165

    accuracy                          0.856      1334
   macro avg      0.668     0.652     0.659      1334
weighted avg      0.849     0.856     0.853      1334

auc macro 0.809
confusion matrix
[[  64  105]
 [  87 1078]]
Model rank: 1
Mean validation score: 0.701 (std: 0.007)
Parameters: {'model__class_weight': 'balanced', 'mod

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced',
                                        criterion='entropy',
                                        max_features='sqrt', min_samples_leaf=4,
                                        n_estimators=158))])

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

hyperparams = {
    'model__n_estimators': stats.randint(10, 100),
    'model__learning_rate': stats.uniform(0.2, 1)
}

model = AdaBoostClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="adaboost")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.623     0.305     0.410       505
         1.0      0.906     0.973     0.939      3494

    accuracy                          0.889      3999
   macro avg      0.765     0.639     0.674      3999
weighted avg      0.871     0.889     0.872      3999

auc macro 0.852
confusion matrix
[[ 154  351]
 [  93 3401]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.506     0.231     0.317       169
         1.0      0.897     0.967     0.931      1165

    accuracy                          0.874      1334
   macro avg      0.702     0.599     0.624      1334
weighted avg      0.847     0.874     0.853      1334

auc macro 0.809
confusion matrix
[[  39  130]
 [  38 1127]]
Model rank: 1
Mean validation score: 0.659 (std: 0.001)
Parameters: {'model__learning_rate': 1.1141167568543

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 AdaBoostClassifier(learning_rate=1.1141167568543486,
                                    n_estimators=20))])

In [12]:
from sklearn.neural_network import MLPClassifier
import random

hyperparams = {
    'model__hidden_layer_sizes': [[stats.randint.rvs(100, 300), stats.randint.rvs(50, 150)], [stats.randint.rvs(50, 300)]], 
    'model__solver': ['sgd', 'adam'], #sgd’ refers to stochastic gradient descent. ‘adam’ refers to a stochastic gradient-based optimizer
    'model__learning_rate_init': stats.uniform(0.0005, 0.005), 
    'model__learning_rate': ('constant', 'adaptive'), 
    'model__alpha': stats.uniform(0, 1), #Strength of the L2 regularization term
    'model__early_stopping': [True],
    'model__max_iter': stats.randint(300, 500),
}

model = MLPClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="nn")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.730     0.230     0.349       505
         1.0      0.899     0.988     0.941      3494

    accuracy                          0.892      3999
   macro avg      0.814     0.609     0.645      3999
weighted avg      0.877     0.892     0.866      3999

auc macro 0.842
confusion matrix
[[ 116  389]
 [  43 3451]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.648     0.207     0.314       169
         1.0      0.895     0.984     0.937      1165

    accuracy                          0.885      1334
   macro avg      0.772     0.595     0.626      1334
weighted avg      0.864     0.885     0.858      1334

auc macro 0.818
confusion matrix
[[  35  134]
 [  19 1146]]
Model rank: 1
Mean validation score: 0.678 (std: 0.017)
Parameters: {'model__alpha': 0.8623563043107623, 'mo

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 MLPClassifier(alpha=0.8623563043107623, early_stopping=True,
                               hidden_layer_sizes=[241, 96],
                               learning_rate_init=0.005252745352124241,
                               max_iter=408))])

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

hyperparams = {
    'model__learning_rate': stats.uniform(0.03, 0.2),
    'model__n_estimators': stats.randint(10, 100),
    'model__max_depth': stats.randint(2, 6),
    'model__max_features': ('sqrt', 'log2', None),  # regularization
    'model__subsample': (0.25, 0.5, 0.75, 1),       # regularization
}

model = GradientBoostingClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="gb")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.989     0.697     0.818       505
         1.0      0.958     0.999     0.978      3494

    accuracy                          0.961      3999
   macro avg      0.973     0.848     0.898      3999
weighted avg      0.962     0.961     0.958      3999

auc macro 0.981
confusion matrix
[[ 352  153]
 [   4 3490]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.468     0.219     0.298       169
         1.0      0.895     0.964     0.928      1165

    accuracy                          0.870      1334
   macro avg      0.682     0.591     0.613      1334
weighted avg      0.841     0.870     0.848      1334

auc macro 0.797
confusion matrix
[[  37  132]
 [  42 1123]]
Model rank: 1
Mean validation score: 0.675 (std: 0.001)
Parameters: {'model__learning_rate': 0.1991917339592

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.19919173395925846,
                                            max_depth=4, n_estimators=98,
                                            subsample=1))])

In [14]:
#Don't run this in jupyter within vscode, run this with notebooks within browsers.
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

hyperparams = {
    'model__booster': ['gbtree', 'gblinear', 'dart'],
    'model__eta': stats.uniform(0.05, 0.5),
    'model__gamma': stats.uniform(0, 0.2),
    'model__max_depth': [2, 3, 4, 6],
    'model__n_estimators': stats.randint(10, 100),
    'model__subsample': [0.25, 0.5, 0.75, 1],     # Stochastic regularization
    'model__lambda': stats.uniform(0.5, 1.5),     # L2 regularization
    'model__alpha': stats.uniform(0, 0.5),        # L1 regularization
    'model__scale_pos_weight': [0.2, 0.4, 0.8, 1, 2],
}

model = xgb.XGBClassifier(n_jobs=1)
train_partial(model=model, hyperparams=hyperparams, savename="xgb")

Fitting 2 folds for each of 5000 candidates, totalling 10000 fits
Testing on training set:
              precision    recall  f1-score   support

         0.0      0.558     0.574     0.566       505
         1.0      0.938     0.934     0.936      3494

    accuracy                          0.889      3999
   macro avg      0.748     0.754     0.751      3999
weighted avg      0.890     0.889     0.889      3999

auc macro 0.892
confusion matrix
[[ 290  215]
 [ 230 3264]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.452     0.450     0.451       169
         1.0      0.920     0.921     0.921      1165

    accuracy                          0.861      1334
   macro avg      0.686     0.685     0.686      1334
weighted avg      0.861     0.861     0.861      1334

auc macro 0.818
confusion matrix
[[  76   93]
 [  92 1073]]
Model rank: 1
Mean validation score: 0.704 (std: 0.003)
Parameters: {'model__alpha': 0.06381730732755597, 'm

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  [1, 25, 17, 2])])),
                ('model',
                 XGBClassifier(alpha=0.06381730732755597,
                               eta=0.2829489268847606,
                               gamma=0.05349053211017592,
                               lambda=1.5428580241591074, n_estimators=69,
                               scale_pos_weight=0.4, subsample=0.5))])

/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=400).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/envs/ml_buzi/lib/python3.8/site-packages/sklearn/svm/_base.p

### Evaluate

In [ ]:
models = [
    "logreg_random_svmsmote_logreg",
    "svc_random_bordersmote_svc",
    "knn2_random_svmsmote_knn2",
    "rf2_random_svmsmote_rf2",
    "adaboost2_random_svmsmote_adaboost2",
    "nn_random_svmsmote_nn",
    "gb_random_svmsmote_gb",
    #"xgb2_random_svmsmote_xgb2",
    #"xgb2",
    #"nn1_random_svmsmote_nn1",
]
name = f"{models[6]}.joblib"
model = load(path_models+name)
#model = load(path_models)
# model.fit(X_train, y_train)
# evaluate(model, X_train, y_train)
print(model)


evaluate(model, X_valid, y_valid)
evaluate(model, X_test, y_test)
X_train.shape

In [ ]:
tree = [load(path_models+f"tree4.joblib")]
evaluate(tree[0], X_valid4, y_valid4)
evaluate(tree[0], X_test4, y_test4)